In [947]:
import pandas as pd
df= pd.read_csv('Loan_Training.csv')
df.head()

,Loan ID,Amount Requested,Amount Funded By Investors,Interest Rate,Loan Length,CREDIT Grade,Loan Title,Loan Purpose,Monthly PAYMENT,Total Amount Funded,...,Inquiries in the Last 6 Months,Accounts Now Delinquent,Delinquent Amount,Delinquencies (Last 2 yrs),Months Since Last Delinquency,Public Records On File,Months Since Last Record,Education,Employment Length,"Status (Fully Paid=1, Not Paid=0)"
0,2227104,12000,12000.00,13.11%,36 months,B4,Credit card refinancing,credit_card,404.97,12000,...,0.0,0.0,0.0,0.0,34.0,0.0,NaN,NaN,7 years,0
1,1690660,2000,2000.00,17.27%,36 months,C5,Car financing,car,71.58,2000,...,3.0,0.0,0.0,0.0,30.0,0.0,NaN,NaN,1 year,1
2,2301695,9000,9000.00,7.90%,36 months,A4,Bundle Loan,debt_consolidation,281.62,9000,...,0.0,0.0,0.0,2.0,10.0,0.0,NaN,NaN,< 1 year,0
3,844352,13000,13000.00,11.99%,36 months,B5,Payback,debt_consolidation,431.73,13000,...,1.0,0.0,0.0,0.0,58.0,0.0,NaN,NaN,5 years,1
4,330773,10000,1525.01,10.71%,36 months,B5,Business Loan,other,163.01,5000,...,1.0,0.0,0.0,0.0,45.0,0.0,NaN,NaN,< 1 year,0


In [948]:
#create a copy of the original data
df1 = df.copy()

#drop columns that have more than 3 N.A values
df1 = df1.dropna(thresh=25)


#delete columns that are irrelevant
del df1['Loan Title']
del df1['Loan ID']
del df1['City']
del df1['Earliest CREDIT Line']
print(df1.shape)
import numpy as np

#changing values of certain columns from percentage (%) to decimal

df1['Interest Rate'] = df1['Interest Rate'].replace('%','',regex=True).astype('float')/100
df1['Debt-To-Income Ratio'] = df1['Debt-To-Income Ratio'].replace('%','',regex=True).astype('float')/100    
df1['Revolving Line Utilization'] = df1['Revolving Line Utilization'].replace('%','',regex=True).astype('float')/100


df1.head()

#normalizing the values of education

df1['Education'] = df1['Education'].replace(np.nan, 'Not Applicable', regex=True)
df1['Education'] = df1['Education'].replace(['n/a','none','None','NONE','undefined'], 'Not Applicable', regex=True)
searchForUni = ['uni','A&M','UCLA','V.C.U','unc','vcu','sdsu','san diego','Penn State','SUNY','Technology','UC','Dartmouth','FMU']
df1.loc[df1['Education'].str.contains('|'.join(searchForUni),case=False),'Education'] = 'University'
searchForCollege = ['college','ccny','NYCCT','coll']
df1.loc[df1['Education'].str.contains('|'.join(searchForCollege),case=False),'Education'] = 'College'
searchForVocational = ['tech','institute','Academy','Navy','vocational']
df1.loc[df1['Education'].str.contains('|'.join(searchForVocational),case=False),'Education'] = 'Vocational/Military School'

getTheRest = ['Not Applicable','University','College','Vocational/Military School']
df1.loc[~df1['Education'].isin(getTheRest),'Education'] = 'High School and Others'  



(19978, 27)


In [952]:
#normalizing values of Employment Length

df1['Employment Length'] = df1['Employment Length'].replace('n/a','< 1 year',regex=True)



array(['7 years', '1 year', '< 1 year', '5 years', '10+ years', '3 years',
       '4 years', '6 years', '2 years', '8 years', '9 years'], dtype=object)

In [954]:
#changing values of FICO RANGE : NaN into mode
df1['FICO Range'] = df1['FICO Range'].replace(np.nan, '680-684', regex=True)

680-684    1340
685-689    1288
675-679    1261
670-674    1244
690-694    1204
665-669    1107
695-699    1092
700-704    1059
705-709     973
660-664     965
710-714     797
725-729     746
715-719     743
720-724     742
730-734     608
735-739     566
740-744     538
750-754     476
745-749     421
755-759     390
760-764     365
770-774     278
765-769     277
775-779     239
780-784     203
790-794     161
785-789     143
795-799     130
655-659      93
800-804      90
650-654      86
640-644      85
645-649      84
805-809      75
810-814      58
815-819      18
820-824      14
825-829       9
830-834       6
840-844       3
835-839       1
Name: FICO Range, dtype: int64

In [956]:
#helper functions to generate random numbers that follow a normal distribution with mean and sigma determined from the exisiting
#values inthe column
def NaNgenerateRandom(df):
    mu = df.mean()
    std = df.std()
    a = df.values
    m = np.isnan(a) #no. of NANs
    a[m] = np.random.normal(mu,std,size=m.sum())
    a[m] = [int(round(x)) for x in a[m]]
    a[m] = [abs(x) for x in a[m]]
    return df
#to ensure that 0.5 does not get rounded up to 1
def NaNgenerateRandomPercent(df):
    mu = df.mean()
    std = df.std()
    a = df.values
    m = np.isnan(a) #no. of NANs
    a[m] = np.random.normal(mu,std,size=m.sum())
    a[m] = [abs(x) for x in a[m]]
    return df

In [957]:
#more normalizing
df1['Months Since Last Record'] = NaNgenerateRandom(df1['Months Since Last Record'])

In [959]:
#Randomize NaN with 0-5 based on their respective probabilities in the columns.
odds = [0.95405,0.04445,0.0014,0.00005,0.00005]
df1['Public Records On File'] = df1['Public Records On File'].replace(np.nan,np.random.choice(np.arange(0, 5), p=odds),regex=True)

In [961]:
df1['Months Since Last Delinquency'] = NaNgenerateRandom(df1['Months Since Last Delinquency'])


In [963]:
df1['Delinquencies (Last 2 yrs)'] = NaNgenerateRandom(df1['Delinquencies (Last 2 yrs)'])

In [965]:
#randomize based on existing probabilities of the values in the column
df1['Delinquent Amount'].value_counts()
odds = ['0'] * 19968 +['334'] * 1 + ['3941.0'] * 1
df1['Delinquent Amount'] = df1['Delinquent Amount'].replace(np.nan,np.random.choice(odds),regex=True)


In [966]:
df1['Revolving Line Utilization'] = NaNgenerateRandomPercent(df1['Revolving Line Utilization'])

In [967]:
df1['Accounts Now Delinquent'].value_counts()
odds = [0.99,0.01]
df1['Accounts Now Delinquent'] = df1['Accounts Now Delinquent'].replace(np.nan,np.random.choice(np.arange(0, 2), p=odds),regex=True)

In [968]:
df1['Revolving CREDIT Balance'] = NaNgenerateRandom(df1['Revolving CREDIT Balance'])

In [969]:

df1['Inquiries in the Last 6 Months'] = NaNgenerateRandom(df1['Inquiries in the Last 6 Months'])


In [970]:
df1['Total CREDIT Lines'] = NaNgenerateRandom(df1['Total CREDIT Lines'])

In [971]:
df1['Open CREDIT Lines'] = NaNgenerateRandom(df1['Open CREDIT Lines'])

In [972]:
#ensure no negative values
df1.loc[(df1['Monthly Income']< 0,'Monthly Income')] = 0

In [973]:
df1['Debt-To-Income Ratio'] = NaNgenerateRandomPercent(df1['Debt-To-Income Ratio'])
df1['Interest Rate'] = NaNgenerateRandomPercent(df1['Interest Rate'])

In [974]:
df1.loc[(df1['Total Amount Funded']< 0, 'Total Amount Funded')] = 0

In [975]:
df1.loc[(df1['Monthly PAYMENT']< 0, 'Monthly PAYMENT')] = 0

In [976]:
df1.loc[(df1['Amount Requested']< 0, 'Amount Requested')] = 0
df1.loc[(df1['Amount Funded By Investors']< 0, 'Amount Funded By Investors')] = 0

36 months    16703
60 months     3275
Name: Loan Length, dtype: int64

In [977]:
#get rid of the 'months' behind 36 and 60
odds = ['36 months'] * 16725 + ['60 months'] * 3275
df1['Loan Length'] = df1['Loan Length'].replace(np.nan,np.random.choice(odds),regex=True)
df1['Loan Length'] = df1['Loan Length'].replace('months','',regex=True)


array(['36 ', '60 '], dtype=object)

In [ ]:
#write to file

In [980]:
df1.to_csv('cleanTrain.csv', encoding='utf-8',index= False)

,Amount Requested,Amount Funded By Investors,Interest Rate,Loan Length,CREDIT Grade,Loan Purpose,Monthly PAYMENT,Total Amount Funded,Debt-To-Income Ratio,State,...,Inquiries in the Last 6 Months,Accounts Now Delinquent,Delinquent Amount,Delinquencies (Last 2 yrs),Months Since Last Delinquency,Public Records On File,Months Since Last Record,Education,Employment Length,"Status (Fully Paid=1, Not Paid=0)"
0,12000,12000.00,0.1311,36,B4,credit_card,404.97,12000,0.1524,CA,...,0.0,0.0,0.0,0.0,34.0,0.0,115.0,Not Applicable,7 years,0
1,2000,2000.00,0.1727,36,C5,car,71.58,2000,0.1207,FL,...,3.0,0.0,0.0,0.0,30.0,0.0,59.0,Not Applicable,1 year,1
2,9000,9000.00,0.0790,36,A4,debt_consolidation,281.62,9000,0.2001,GA,...,0.0,0.0,0.0,2.0,10.0,0.0,107.0,Not Applicable,< 1 year,0
3,13000,13000.00,0.1199,36,B5,debt_consolidation,431.73,13000,0.0759,MN,...,1.0,0.0,0.0,0.0,58.0,0.0,8.0,Not Applicable,5 years,1
4,10000,1525.01,0.1071,36,B5,other,163.01,5000,0.1248,NV,...,1.0,0.0,0.0,0.0,45.0,0.0,4.0,Not Applicable,< 1 year,0
